<div style='text-align: center; margin-bottom: 40px;'>
    <h1>🛰️ SeekerLight</h1>
    <h3>Satellite Imagery Analysis Platform</h3>
</div>

<div style='margin-bottom: 20px;'>
Welcome to SeekerLight! This application helps you analyze Sentinel-1 SAR (Synthetic Aperture Radar) imagery.

### Features:
- View SAR data in VH or VV polarization
- Filter by orbit pass (Ascending/Descending)
- Draw and save custom Areas of Interest (AOIs)
- Analyze temporal changes
</div>

In [ ]:
# Parameters and imports (this cell will be hidden)
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime, timedelta
import os
import warnings
import shutil

# Suppress warnings
warnings.filterwarnings('ignore')

# Initialize Earth Engine
try:
    ee.Initialize(project='ee-sergiyk1974')
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Initialize Sentinel-1 collection
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')

# File to store AOIs
AOI_MASTER_FILE = 'custom_aois_master.py'

# Copy AOIs from sat-scanner if our file doesn't exist
if not os.path.exists(AOI_MASTER_FILE):
    src_file = '../sat-scanner/custom_aois_master.py'
    if os.path.exists(src_file):
        shutil.copy2(src_file, AOI_MASTER_FILE)

# Function to read existing AOIs
def read_existing_aois():
    if not os.path.exists(AOI_MASTER_FILE):
        return {}
    try:
        namespace = {'ee': ee}
        with open(AOI_MASTER_FILE) as f:
            exec(f.read(), namespace)
        return namespace.get('CUSTOM_AOIS', {})
    except Exception as e:
        print(f"Error reading AOIs: {str(e)}")
        return {}

# Function to write AOIs to file
def write_aois_file(aois_dict):
    content = [
        'import ee\n',
        '# Master file for custom AOIs\n',
        'CUSTOM_AOIS = {\n'
    ]
    
    for name, aoi_info in sorted(aois_dict.items()):
        coords_str = str(aoi_info['coords']).replace('], [', '],\n            [')
        content.extend([
            f"    '{name}': {{\n",
            f"        'coords': {coords_str},\n",
            f"        'description': '{aoi_info.get('description', '')}',\n",
            f"        'area_km2': {aoi_info.get('area_km2', 0):.2f}\n",
            "    },\n"
        ])
    
    content.extend([
        "}\n\n",
        "# Create ee.Geometry objects for each AOI\n",
        "for name, aoi_info in CUSTOM_AOIS.items():\n",
        "    aoi_info['geometry'] = ee.Geometry.Polygon([aoi_info['coords']])\n"
    ])
    
    with open(AOI_MASTER_FILE, 'w') as f:
        f.writelines(content)

# Initialize AOIs
aois = read_existing_aois()
print("Loaded AOIs:", list(aois.keys()))

In [ ]:
# Create the application interface
title = HTML('<div style="text-align:center; margin: 20px 0;"><h2>Satellite Data Viewer</h2></div>')
display(title)

# Create map
m = geemap.Map(
    center=[47.2357, 39.7015],
    zoom=8,
    height='600px',
    layout=widgets.Layout(width='800px', height='600px')
)

# Add basemap
m.add_basemap('HYBRID')

# Add drawing controls
m.add_draw_control()

# Create widgets
aoi_selector = widgets.Dropdown(
    options=list(aois.keys()),
    description='Area of Interest:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Create date range widgets
start_date = widgets.DatePicker(
    description='Start Date:',
    value=datetime.now() - timedelta(days=30),
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

end_date = widgets.DatePicker(
    description='End Date:',
    value=datetime.now(),
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

band_select = widgets.Dropdown(
    options=['VH', 'VV'],
    value='VH',
    description='Polarization:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

orbit_select = widgets.Dropdown(
    options=['BOTH', 'ASCENDING', 'DESCENDING'],
    value='BOTH',
    description='Orbit Pass:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

update_button = widgets.Button(
    description='Update View',
    button_style='primary',
    icon='refresh',
    layout=widgets.Layout(width='150px')
)

save_aoi_button = widgets.Button(
    description='Save Current AOI',
    button_style='success',
    icon='save',
    layout=widgets.Layout(width='150px')
)

output = widgets.Output(
    layout=widgets.Layout(width='300px', height='100px')
)

# Layout widgets
controls = widgets.VBox([
    widgets.HTML('<h3>Controls</h3>'),
    aoi_selector,
    widgets.HBox([
        widgets.VBox([
            widgets.HTML('<b>Date Range</b>'),
            start_date,
            end_date
        ]),
        widgets.VBox([
            widgets.HTML('<b>Settings</b>'),
            band_select,
            orbit_select
        ])
    ], layout=widgets.Layout(justify_content='space-between')),
    widgets.HBox([update_button, save_aoi_button], 
                layout=widgets.Layout(justify_content='space-between', margin='10px 0')),
    widgets.HTML('<h3>Status</h3>'),
    output
], layout=widgets.Layout(width='350px', margin='0 0 0 20px'))

# Create main layout
main_layout = widgets.HBox([
    m,
    controls
], layout=widgets.Layout(
    justify_content='center',
    align_items='flex-start',
    width='100%'
))

# Display layout
display(main_layout)

# Event handlers
def on_update_click(b):
    with output:
        clear_output()
        try:
            if not aoi_selector.value:
                print("Please select an Area of Interest")
                return
                
            # Get AOI
            aoi_info = aois[aoi_selector.value]
            geometry = ee.Geometry.Polygon([aoi_info['coords']])
            
            # Get date range
            start = start_date.value.strftime('%Y-%m-%d')
            end = end_date.value.strftime('%Y-%m-%d')
            
            # Filter collection
            collection = sentinel1.filterBounds(geometry) \
                .filterDate(start, end)
            
            if orbit_select.value != 'BOTH':
                collection = collection.filter(
                    ee.Filter.eq('orbitProperties_pass', orbit_select.value)
                )
            
            # Create composite
            composite = collection.select(band_select.value).mean().clip(geometry)
            
            # Set visualization parameters
            vis_params = {
                'min': -25,
                'max': 0,
                'palette': ['black', 'white']
            }
            
            # Clear existing layers and add new one
            m.layers = m.layers[:2]  # Keep only basemaps
            m.addLayer(composite, vis_params, f'SAR {band_select.value}')
            
            # Center map on AOI
            m.centerObject(geometry)
            
            print("View updated successfully")
            
        except Exception as e:
            print(f"Error updating view: {str(e)}")

def on_save_aoi(b):
    with output:
        clear_output()
        try:
            # Get drawn features
            drawn_features = m.draw_features
            if not drawn_features:
                print("Please draw an AOI first using the drawing tools")
                return
                
            # Get the last drawn feature
            feature = drawn_features[-1]
            if feature['geometry']['type'] != 'Polygon':
                print("Please draw a polygon")
                return
                
            # Get coordinates
            coords = feature['geometry']['coordinates'][0]
            
            # Calculate area
            geometry = ee.Geometry.Polygon([coords])
            area_km2 = geometry.area().getInfo() / 1_000_000
            
            # Create AOI name
            base_name = "Custom AOI"
            name = base_name
            counter = 1
            while name in aois:
                name = f"{base_name} {counter}"
                counter += 1
            
            # Add to AOIs
            aois[name] = {
                'coords': coords,
                'description': 'Created from AOI Manager',
                'area_km2': area_km2
            }
            
            # Save to file
            write_aois_file(aois)
            
            # Update dropdown
            aoi_selector.options = list(aois.keys())
            aoi_selector.value = name
            
            print(f"AOI saved as '{name}' (Area: {area_km2:.2f} km²)")
            
        except Exception as e:
            print(f"Error saving AOI: {str(e)}")

# Connect event handlers
update_button.on_click(on_update_click)
save_aoi_button.on_click(on_save_aoi)

# Initial update
if aois:
    on_update_click(None)